In [59]:
%time !python ../ice_hold/both-jails.py

requesting for city inmates in the past 90 days
requesting dlm moss inmates
cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.cty.cty.cty.cty.cty.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.dlm.cty.cty.

In [60]:
import numpy as np
import pandas as pd


county = pd.read_csv('data/dlm_inmates.csv')

city = pd.read_csv('data/tulsa_city_inmates.csv')



## Combine records

In [61]:
# make standard field names

standard_fields = ['FirstName', 'LastName', 'id', 'Gender', 'BookingDate', 'Hold', 'ReleaseDate', 'Race', 'Charge']

county_fields = ['First', 'Last', 'DLM', 'Gender', 'Booking Date', 'TLS', 'Release Date', 'Race', 'Description']

city_fields = ['firstName', 'lastName', 'prisonerID', 'gender', 'bookingDateTime', 'hold', 'releasedDateTime', 'race', 'firstCharge']


In [62]:

county_inmates = pd.DataFrame(columns = standard_fields)
city_inmates = pd.DataFrame(columns = standard_fields)

for idx,val in enumerate(standard_fields):
    county_inmates[val] = county[county_fields[idx]]
    city_inmates[val] = city[city_fields[idx]]

county_inmates['Jail'] = 'DL Moss'
city_inmates['Jail'] = 'City of Tulsa'

frames = [county_inmates, city_inmates]

all_inmates = pd.concat(frames)
    
all_inmates.head()

,FirstName,LastName,id,Gender,BookingDate,Hold,ReleaseDate,Race,Charge,Jail
0,DENIS,CHEN-VELAZCO,1260807,M,1/27/2019,NaN,NaN,W,HOLD/ICE,DL Moss
1,OSCAR,MARQUEZ AGUILAR,1260806,M,1/27/2019,NaN,NaN,O,HOLD/ICE,DL Moss
2,CHOWDHURY,RAYHAN,1260805,M,1/27/2019,NaN,NaN,O,HOLD/ICE,DL Moss
3,RAHMAN,FAZLUR,1260804,M,1/27/2019,NaN,NaN,W,HOLD/ICE,DL Moss
4,MOHAMMAD,RASHED KHAN,1260803,M,1/27/2019,NaN,NaN,O,HOLD/ICE,DL Moss


## Clean up

In [63]:
# use the same gender value
gender_translate ={"F":"F","Female":"F", "Male": "M", "M":"M"}
gender = lambda str: gender_translate.get(str,'unknown')
all_inmates['Gender'] = all_inmates['Gender'].apply(gender)
all_inmates.groupby('Gender').count()

,FirstName,LastName,id,BookingDate,Hold,ReleaseDate,Race,Charge,Jail
Gender,,,,,,,,,
F,655,655,655,655,56,405,655,654,655
M,2137,2137,2137,2137,176,939,2135,2126,2137


In [64]:
# use the same race values
race_translate ={
        "Asian" : "Asian",
        "B" : "Black",
        "Black" : "Black",
        "H" : "Hispanic",
        "Hispanic" : "Hispanic",
        "I" : "Indian",
        "Indian" : "Indian",
        "O" : "Asian",
        "Unk" : "Unknown",
        "Unknown" : "Unknown",    
        "W" : "White",
        "White": "White"
    }
race = lambda str: race_translate.get(str,'Unknown')
all_inmates['Race'] = all_inmates['Race'].apply(race)
all_inmates.groupby('Race').count()

,FirstName,LastName,id,Gender,BookingDate,Hold,ReleaseDate,Charge,Jail
Race,,,,,,,,,
Asian,46,46,46,46,46,1,4,46,46
Black,904,904,904,904,904,61,482,900,904
Hispanic,101,101,101,101,101,56,7,101,101
Indian,116,116,116,116,116,8,63,115,116
Unknown,12,12,12,12,12,1,10,10,12
White,1613,1613,1613,1613,1613,105,778,1608,1613


In [65]:
# convert the dates from a string to a python DateTime type
from datetime import datetime

def pd_to_datetime(date_value):
    return pd.to_datetime(date_value) if not pd.isna(date_value) else  datetime(1970,1,1,0,0)

all_inmates['BookingDate'] = all_inmates['BookingDate'].apply(pd_to_datetime)
all_inmates['ReleaseDate'] = all_inmates['ReleaseDate'].apply(pd_to_datetime)


In [66]:
# convert null hold to str 

def safe_hold(hold_val):
    return hold_val if not pd.isna(hold_val) else "No hold"

all_inmates['Hold'] = all_inmates['Hold'].apply(safe_hold)

all_inmates.groupby('Hold').count()

,FirstName,LastName,id,Gender,BookingDate,ReleaseDate,Race,Charge,Jail
Hold,,,,,,,,,
Hold for Agency,2,2,2,2,2,2,2,2,2
Hold for Agency Carter County,1,1,1,1,1,1,1,1,1
Hold for Agency Creek County,1,1,1,1,1,1,1,1,1
Hold for Agency ICE,2,2,2,2,2,2,2,2,2
Hold for Agency Mayes County Sherriff Department,1,1,1,1,1,1,1,1,1
Hold for Agency Mental Health,5,5,5,5,5,5,5,5,5
Hold for Agency Muskogee County,1,1,1,1,1,1,1,1,1
Hold for Agency OCSO,1,1,1,1,1,1,1,1,1
Hold for Agency Osage County,1,1,1,1,1,1,1,1,1


In [67]:
# fix null charge str

def safe_charge(chg_val):
    return chg_val if not pd.isna(chg_val) else "No charge listed"

all_inmates['Charge'] = all_inmates['Charge'].apply(safe_charge)

all_inmates.groupby('Charge').count()

# safety check for null names

def safe_name(name_str):
    return name_str if not pd.isna(name_str) else "Not_entered"


all_inmates['FirstName'] = all_inmates['FirstName'].apply(safe_name)
all_inmates['LastName'] = all_inmates['LastName'].apply(safe_name)


In [68]:
# sort by Booking date so that last booking is best record
sorted_inmates = all_inmates.sort_values('BookingDate')
sorted_inmates

,FirstName,LastName,id,Gender,BookingDate,Hold,ReleaseDate,Race,Charge,Jail
1628,MICHAEL,BROWNING,1101280,M,2013-06-24 00:00:00,No hold,1970-01-01 00:00:00,White,HOLD/RETURN TO OKLAHOMASTATE PENITENTIARY,DL Moss
1627,MICHAEL,RUBLE,1159942,M,2013-08-08 00:00:00,No hold,1970-01-01 00:00:00,Black,A&B W/INTENT TO COMMIT FELONY,DL Moss
1626,BRANDE,SAMUELS,9603041,M,2016-03-30 00:00:00,No hold,1970-01-01 00:00:00,Black,SHOOT W/INTENT TO KILL AFCF,DL Moss
1625,DANIEL,URQUIZA,1123503,M,2016-04-06 00:00:00,TLS1604011,1970-01-01 00:00:00,Hispanic,TRAFFIC METHAMPHETAMINE,DL Moss
1624,ABRAHAM,SANCHEZ,1241585,M,2016-04-06 00:00:00,TLS1604012,1970-01-01 00:00:00,Hispanic,TRAFFIC METHAMPHETAMINE,DL Moss
1623,JUAN,URQUIZA-PERALTA,1224665,M,2016-04-06 00:00:00,TLS1604014,1970-01-01 00:00:00,Hispanic,AGGRAVATED TRAFFIC ILLEGAL DRUGS,DL Moss
1622,JASON,BOOKER,1242398,M,2016-05-17 00:00:00,No hold,1970-01-01 00:00:00,White,SEXUAL ABUSE MINOR,DL Moss
1621,JACKY,MAYFIELD,1154367,M,2016-06-17 00:00:00,No hold,1970-01-01 00:00:00,Black,MURDER 1ST DEGREE,DL Moss
1620,HENRY,KWEKWE,1243212,M,2016-07-28 00:00:00,No hold,1970-01-01 00:00:00,Black,CONJOINT ROBBERY,DL Moss
1619,BRENNON,LOVETT,1007586,M,2016-07-29 00:00:00,No hold,1970-01-01 00:00:00,Black,SHOOT W/INTENT TO KILL,DL Moss


In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es = Elasticsearch(http_compress=True)

keys = ['FirstName', 'LastName', 'id', 'Gender', 'BookingDate', 'Hold', 'ReleaseDate', 'Race', 'Charge', 'Jail']


def filterKeys(doc):
    return {k: doc[k] for k in keys }

def doc_generator(df):
    df_iter = df.iterrows()
    for idx, document in df_iter:
        yield {
                "_index": 'inmates3',
                "_type": "_doc",
                "_id" : f"{document['id']}",
                "_source": filterKeys(document),
            }
    raise StopIteration

helpers.bulk(es, doc_generator(sorted_inmates))

ModuleNotFoundError: No module named 'elasticsearch'

In [74]:
df_iter = sorted_inmates.iterrows()

idx, document = next(df_iter)
document.to_dict()

{'FirstName': 'MICHAEL',
 'LastName': 'BROWNING',
 'id': 1101280,
 'Gender': 'M',
 'BookingDate': Timestamp('2013-06-24 00:00:00'),
 'Hold': 'No hold',
 'ReleaseDate': Timestamp('1970-01-01 00:00:00'),
 'Race': 'White',
 'Charge': 'HOLD/RETURN TO OKLAHOMASTATE PENITENTIARY',
 'Jail': 'DL Moss'}

In [71]:
type(document)


pandas.core.series.Series